In [1]:
import os
from snowflake.snowpark import Session
import sys
import logging



In [2]:
# initiate logging at info level
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%I:%M:%S')



In [3]:
# snowpark session
def get_snowpark_session() -> Session:
    connection_parameters = {
       "ACCOUNT":"account",
        "USER":"user",
        "PASSWORD":"password",
        "ROLE":"SYSADMIN",
        "DATABASE":"SALES_DWH",
        "SCHEMA":"SOURCE",
        "WAREHOUSE":"SNOWPARK_ETL_WH"
    }
    # creating snowflake session object
    return Session.builder.configs(connection_parameters).create()

In [4]:
session = get_snowpark_session()

context_df = session.sql("select current_role(), current_database(), current_schema(), current_warehouse()")
context_df.show(2)

10:08:41 - INFO - Snowflake Connector for Python Version: 3.7.0, Python Version: 3.8.8, Platform: macOS-10.16-x86_64-i386-64bit
10:08:41 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
10:08:42 - INFO - Snowpark Session information: 
"version" : 1.13.0,
"python.version" : 3.8.8,
"python.connector.version" : 3.7.0,
"python.connector.session.id" : 1276855122608270,
"os.name" : Darwin

10:08:42 - INFO - query: [select current_role(), current_database(), current_schema(), current_warehouse()...]
10:08:42 - INFO - query execution done
10:08:42 - INFO - Number of results in first chunk: 0
10:08:42 - INFO - query: [SELECT  *  FROM (select current_role(), current_database(), current_schema(), cu...]
10:08:42 - INFO - query execution done
10:08:42 - INFO - Number of results in first chunk: 1
------

In [17]:

def traverse_directory(directory,file_extension) -> list:
    local_file_path = []
    file_name = []  # List to store CSV file paths
    partition_dir = []
    print(directory)
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(file_extension):
                file_path = os.path.join(root, file)
                file_name.append(file)
                partition_dir.append(root.replace(directory, ""))
                local_file_path.append(file_path)

    return file_name,partition_dir,local_file_path


In [18]:
def main():
    # Specify the directory path to traverse
    directory_path = '/Users/rajphanindrakakarla/Downloads/SF_Py_Project/Datasets/'
    csv_file_name, csv_partition_dir , csv_local_file_path= traverse_directory(directory_path,'.csv')
    parquet_file_name, parquet_partition_dir , parquet_local_file_path= traverse_directory(directory_path,'.parquet')
    json_file_name, json_partition_dir , json_local_file_path= traverse_directory(directory_path,'.json')
    stage_location = '@sales_dwh.source.my_internal_stg'

    
    csv_index = 0
    for file_element in csv_file_name:
        put_result = ( 
                    get_snowpark_session().file.put( 
                        csv_local_file_path[csv_index], 
                        stage_location+"/"+csv_partition_dir[csv_index], 
                        auto_compress=False, overwrite=True, parallel=10)
                    )
        #put_result(file_element," => ",put_result[0].status)
        csv_index+=1

    parquet_index = 0
    for file_element in parquet_file_name:

        put_result = ( 
                    get_snowpark_session().file.put( 
                        parquet_local_file_path[parquet_index], 
                        stage_location+"/"+parquet_partition_dir[parquet_index], 
                        auto_compress=False, overwrite=True, parallel=10)
                    )
        #put_result(file_element," => ",put_result[0].status)
        parquet_index+=1
        json_index = 0
    for file_element in parquet_file_name:

        put_result = ( 
                    get_snowpark_session().file.put( 
                        json_local_file_path[json_index], 
                        stage_location+"/"+json_partition_dir[json_index], 
                        auto_compress=False, overwrite=True, parallel=10)
                    )
        #put_result(file_element," => ",put_result[0].status)
        json_index+=1  
    
    
if __name__ == '__main__':
    main()

/Users/rajphanindrakakarla/Downloads/SF_Py_Project/Datasets/
/Users/rajphanindrakakarla/Downloads/SF_Py_Project/Datasets/
/Users/rajphanindrakakarla/Downloads/SF_Py_Project/Datasets/
05:10:09 - INFO - Snowflake Connector for Python Version: 3.7.0, Python Version: 3.8.8, Platform: macOS-10.16-x86_64-i386-64bit
05:10:09 - INFO - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
05:10:10 - INFO - Snowpark Session information: 
"version" : 1.13.0,
"python.version" : 3.8.8,
"python.connector.version" : 3.7.0,
"python.connector.session.id" : 1276855122542782,
"os.name" : Darwin

05:10:10 - INFO - query: [PUT 'file:///Users/rajphanindrakakarla/Downloads/SF_Py_Project/Datasets/sales/so...]
05:10:10 - INFO - query execution done
05:10:10 - INFO - Snowflake Connector for Python Version: 3.7.0, Python Version: 

IndexError: list index out of range